<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

zsh:1: no matches found: https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1


In [1]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [2]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [3]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [4]:
# method most_simmilar
'''your code'''
wv_embeddings.most_similar(wv_embeddings[word])

[('dog', 1.0),
 ('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.7607938647270203),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315489053726196),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191)]

**Ответ: если приравнять cat и cats, то это слово 5 месте в топе самых близких слов; в противном случае считаем, что нет слова в топе похожих**

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [5]:
# import numpy as np
# import re
# # you can use your tokenizer
# # for example, from nltk.tokenize import WordPunctTokenizer
# class MyTokenizer:
#     def __init__(self):
#         pass
#     def tokenize(self, text):
#         return re.findall('\w+', text)
# tokenizer = MyTokenizer()

In [6]:
import numpy as np

In [7]:
from nltk.corpus import stopwords
import re
import spacy
from nltk.stem import PorterStemmer, SnowballStemmer
class MyTokenizer:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.nlp = spacy.load("en_core_web_sm")
    def tokenize(self, text):
        tokenized_sent = [word for word in re.findall('\w+', text.lower()) if word not in self.stop_words]
        return tokenized_sent
tokenizer = MyTokenizer()

# class MyTokenizer:
#     def __init__(self):
#         self.stop_words = set(stopwords.words('english'))
#         self.nlp = spacy.load("en_core_web_sm")
#     def tokenize(self, text):
#         tokenized_sent = [self.nlp(word)[0].lemma_ for word in re.findall('\w+', text.lower()) if word not in self.stop_words]
#         return tokenized_sent
# tokenizer = MyTokenizer()

# class MyTokenizer:
#     def __init__(self):
#         self.stop_words = set(stopwords.words('english'))
#         self.ss = SnowballStemmer(language="english")
#     def tokenize(self, text):
#         tokenized_sent = [self.ss.stem(word) for word in re.findall('\w+', text.lower()) if word not in self.stop_words]
#         return tokenized_sent
# tokenizer = MyTokenizer()

In [8]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''
    counter = 0
    tokenized_question = tokenizer.tokenize(question)
    vectorized_sentence = np.zeros(dim)
    for token in tokenized_question:
        if token in embeddings:
            vectorized_sentence += embeddings[token]
            counter += 1 # we take into account only the words in embedding dictionary
    if counter != 0:
        return vectorized_sentence / counter
    else:
        return vectorized_sentence

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [9]:
'''your code'''
print(round(question_to_vec("I love neural networks", wv_embeddings, tokenizer)[2], 2))

-1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Максимизация данного выражение достигается максимизацией Hits@47 и минимизацией DCG@1

1) Минимальное значение DCG@1 равно 0 (получается в том случае, когда ни один дубликат каждого из N предложений не попал на 1 место)

2) Максимальное значение Hits@47 равно 1 (в случае, когда для всех N предложений дубликат попал в топ 47 в ранжированном списке)

**Ответ:** 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

**Ответ:** $\frac{1}{log_{2}(1+9)}\cdot[9\le10] = 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [10]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    hits_value = sum(np.array(dup_ranks) <= k)
    return hits_value    

In [11]:
import math
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dcg_value = sum(list(map(lambda rating: (rating <= k) / math.log2(1+rating), dup_ranks)))
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [12]:
import pandas as pd

In [13]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
short_vestorizer = lambda x: question_to_vec(x, wv_embeddings, tokenizer)
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [np.array([wv_embeddings.cosine_similarities(short_vestorizer(copy_answers[0]), 
                                                        np.expand_dims(short_vestorizer(simularity), 1).T) 
                      for simularity in candidates_ranking[0]]).argmax() + 1]# search of top-1 simular sentence
dup_ranks = [2] # we can find simular sentence by ourselfs and check out metrics

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [14]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [15]:
# !unzip stackoverflow_similar_questions.zip

Считайте данные.

In [16]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split("\t"))#'''your code'''
    return data

Нам понадобиться только файл validation.

In [17]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [18]:
len(validation_data)

3760

Размер нескольких первых строк

In [19]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [46]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    vectorized_question = question_to_vec(question, embeddings, tokenizer, dim).reshape(-1, 1).T
    simularity_dict = {}
    for candidat_info in enumerate(candidates):
        simularity_dict[candidat_info] = cosine_similarity(vectorized_question, 
                                                           question_to_vec(candidat_info[1], 
                                                                           embeddings, 
                                                                           tokenizer, 
                                                                           dim).reshape(-1, 1).T).item()
    ranked_array = list(dict(sorted(simularity_dict.items(), 
                                    key=lambda item: item[1], 
                                    reverse=True)).keys())
    return ranked_array

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [47]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [48]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [25]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

SyntaxError: invalid syntax (1724534416.py, line 5)

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ: 021**

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [56]:
from tqdm.notebook import tqdm

In [50]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [51]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, 
                                              dcg_score(wv_ranking, k) / max_validation_examples, 
                                              k, 
                                              hits_count(wv_ranking, k) / max_validation_examples))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.416 | Hits@   1: 0.416
DCG@   5: 0.503 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.571 | Hits@ 100: 0.876
DCG@ 500: 0.584 | Hits@ 500: 0.973
DCG@1000: 0.587 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [52]:
def read_corpus_for_embedding(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        line_wothout_punct = re.findall(r"\w+", line)
        data.append(line_wothout_punct)
    return data

In [53]:
train_data = read_corpus_for_embedding('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [78]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(train_data, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=30,             # consider words that occured at least 5 times
                 window=10).wv 

'\nизменение параметров не особо улучшило \nкачество модели, поэтому оставил такие\n'

In [79]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [80]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, 
                                              dcg_score(wv_ranking, k) / max_validation_examples, 
                                              k, 
                                              hits_count(wv_ranking, k) / max_validation_examples))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.347 | Hits@   1: 0.347
DCG@   5: 0.427 | Hits@   5: 0.501
DCG@  10: 0.453 | Hits@  10: 0.580
DCG@ 100: 0.501 | Hits@ 100: 0.811
DCG@ 500: 0.520 | Hits@ 500: 0.957
DCG@1000: 0.524 | Hits@1000: 1.000


**Увеличение размера "контекста" для каждого слова и порога минимальной частоты встречаемости слова повышает качество модели, при этом увеличивается время обработки данных**

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1) Были испробованы несколько подходов токенизации: выделение слов посредством регулярных выражений, регулярки + приведение к нижнему регистру + отбрасывание stopwords, регулярки + нижний регистр + лемматизация + отбрасывание stopwords, регулярки + нижний регистр + стемминг + отбрасывание stopwords. Наилучший результат показал подход "регулярки + приведение к нижнему регистру + отбрасывание stopwords"; лемматизация занимает слишком много времени.

2) В проведенных экспериментах нормализация наоборот ухудшило качество модели(можно, наверное, списать на название домашнего задания "простой эмбеддинг", которое вводит в курс дела). В теории, нормализация должна повысить качество модели.

3) В задании было рассмотрено 2 эмбеддинга: предобученный на корпусе вопросов со stackoverflow и обученный нами. Лучшее качество показал предобученный, это, скорее всего, связано с размером корпуса для этого эмбеддинга. Объема нашей тренировочной выборки по-видимому не хватает для обеспечения такого же качества.

4) В данном задании были использованы простейшие методы для ранжирования схожих вопросов, поэтому не получилось достичь хорошего качества. Так же следовало бы изучить состав тренировочной выборки и тестовой: в тестовой выборки могут быть слова не встречающиеся в тренировочной, а векторное преставление нашего предложения находится как среднее по всем векторам, а новые слова задаются нулевым вектором. Такой подход и приводит к невысокому качеству решения задачи.

5) Регулярки + приведение к нижнему регистру(возможно за исключением общепринятых аббревиатур) + отбрасывание stopwords + лемматизация + предобученный на большом корпусе вопросов stackoverflow эмбеддинг + другое представление предложения(возможно суммирование всех слов с весами + альтернативная обработка не встречавшихся слов)